## 1. Calcolare la probabilità (giornata per giornata)

Per ogni giornata dalla G4 alla G38, calcolare per ogni giocatore:

* **Prob_Gol_Gi**: la probabilità di segnare nella giornata Gi, stimata prima di Gi, usando solo le giornate precedenti

Output atteso: un file CSV per ogni giornata o un unico file finale con righe del tipo:

| Giocatore        | Giornata | Prob\_Gol |
| ---------------- | -------- | --------- |
| Lautaro Martinez | G4       | 0.72      |
| ...              | ...      | ...       |
| Lautaro Martinez | G5       | 0.51      |
| ...              | ...      | ...       |

Strategia: da GolGiocatoriTemporale.csv si estrae la sequenza G1 fino a Gi−1

* Si costruisce la catena di Markov (con Laplace smoothing)

* Si calcola Prob_Gol = somma delle probabilità di transizione verso stato 1 o 2

* Si pesa con i gol segnati fino a Gi−1:
  * Prob_Gol *= log1p(GolFinoGi−1) / 2.5

In [ ]:
import pandas as pd
import numpy as np

# Carica il file dei gol per giornata
df = pd.read_csv('GolGiocatoriTemporale.csv')

# Estrai tutte le giornate (G1–G38)
giornate = [col for col in df.columns if col.startswith('G') and col[1:].isdigit()]
if len(giornate) < 39:
    giornate.append(f'G{len(giornate)+1}')

# Funzione per mappare i gol in stati
def gol_to_stato(x):
    if x == 0: return 0
    elif x == 1: return 1
    else: return 2

# Lista per raccogliere tutte le righe
righe_prob = []

# Ciclo dalla G4 (i=3) fino a G39 (i=38)
for i in range(3, 39):
    Gi = f'G{i+1}'  # Giornata attuale
    giornate_passate = giornate[:i]  # G1...Gi−1 (es. G1–G38 per G39)

    print(f"Calcolo probabilità per {Gi} basata su {giornate_passate}")

    dati_stati = df[giornate_passate].map(gol_to_stato)
    gol_fino_Gi_1 = df[giornate_passate].sum(axis=1)
    peso_gol = np.log1p(gol_fino_Gi_1) / 2.5

    for idx, row in dati_stati.iterrows():
        trans = np.ones((3, 3))  # Laplace smoothing
        for k in range(len(row)-1):
            trans[row.iloc[k]][row.iloc[k+1]] += 1

        trans_somma = trans.sum(axis=1, keepdims=True)
        matrice_prob = trans / trans_somma
        stato_attuale = row.iloc[-1]

        prob = matrice_prob[stato_attuale][1] + matrice_prob[stato_attuale][2]
        trans_count = trans_somma[stato_attuale][0]
        if trans_count < 5:
            prob *= (trans_count / 5)
        prob *= peso_gol.iloc[idx]
        prob = min(prob, 1.0)

        righe_prob.append({
            'Giocatore': df.loc[idx, 'Giocatore'],
            'Giornata': Gi,
            'Prob_Gol': round(prob, 4),
            'GolFinoA_Gi_1': int(gol_fino_Gi_1.iloc[idx])
        })

# Crea il DataFrame finale
df_prob = pd.DataFrame(righe_prob)

# Recupera il nome dell'ultima giornata calcolata (es. 'G39')
ultima_giornata = righe_prob[-1]['Giornata']  # ultima riga inserita

# Costruisci nome del file in base alla giornata finale
nome_file = f"Probabilita_Storica_G4_{ultima_giornata}.csv"

# Salva il file con nome dinamico
df_prob.to_csv(nome_file, index=False)
print(f"File salvato come {nome_file}")

# Controlli su valori anomali di probabilità
print("\nControllo valori anomali:")

# Controllo valori anomali per giornata

# Prob_Gol == 0.0
print("\nProb_Gol == 0.0 per giornata:")
zero_counts = df_prob[df_prob["Prob_Gol"] == 0.0]["Giornata"].value_counts()
zero_counts.index = zero_counts.index.str.extract(r'G(\d+)').astype(int)[0]
zero_counts = zero_counts.sort_index()
for idx, count in zero_counts.items():
    print(f"– G{idx}: {count} casi")

# Prob_Gol ≥ 1.0
print("\nProb_Gol ≥ 1.0 per giornata:")
alta_counts = df_prob[df_prob["Prob_Gol"] >= 1.0]["Giornata"].value_counts()
alta_counts.index = alta_counts.index.str.extract(r'G(\d+)').astype(int)[0]
alta_counts = alta_counts.sort_index()
for idx, count in alta_counts.items():
    print(f"– G{idx}: {count} casi")




Calcolo probabilità per G4 basata su ['G1', 'G2', 'G3']
Calcolo probabilità per G5 basata su ['G1', 'G2', 'G3', 'G4']
Calcolo probabilità per G6 basata su ['G1', 'G2', 'G3', 'G4', 'G5']
Calcolo probabilità per G7 basata su ['G1', 'G2', 'G3', 'G4', 'G5', 'G6']
Calcolo probabilità per G8 basata su ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7']
Calcolo probabilità per G9 basata su ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8']
Calcolo probabilità per G10 basata su ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9']
Calcolo probabilità per G11 basata su ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10']
Calcolo probabilità per G12 basata su ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11']
Calcolo probabilità per G13 basata su ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11', 'G12']
Calcolo probabilità per G14 basata su ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11', 'G12', 'G13']
Calcolo probabilità per G15 basata su [

In [2]:
# Estrai le colonne delle giornate
giornate = [col for col in df.columns if col.startswith('G') and col[1:].isdigit()]
dati = df[giornate].copy()

# Funzione per mappare i gol in stati
def gol_to_stato(x):
    if x == 0: return 0
    elif x == 1: return 1
    else: return 2

# Mappa i gol in stati (0,1,2)
dati_stati = dati.map(gol_to_stato)

#Specifica qui il nome del giocatore da analizzare
nome_giocatore = "lautaro martinez"

# Trova l'indice del giocatore
idx = df[df['Giocatore'].str.lower() == nome_giocatore.lower()].index[0]
row_stati = dati_stati.iloc[idx]
row_gol = dati.iloc[idx]

# Matrice di transizione (Laplace smoothing)
trans = np.ones((3, 3))  # inizializza con 1

for i in range(len(row_stati) - 1):
    stato_corrente = row_stati.iloc[i]
    stato_successivo = row_stati.iloc[i + 1]
    trans[stato_corrente][stato_successivo] += 1

# Matrice di probabilità (normalizzata)
somma_per_riga = trans.sum(axis=1, keepdims=True)
matrice_prob = trans / somma_per_riga

# Stato attuale (ultimo)
stato_attuale = row_stati.iloc[-1]
etichetta_stato = ['0 gol', '1 gol', '2+ gol']

# Calcolo probabilità pura
prob_pura = matrice_prob[stato_attuale][1] + matrice_prob[stato_attuale][2]

# Calcolo peso e probabilità finale
gol_totali = row_gol.sum()
peso = np.log1p(gol_totali) / 2.5
prob_finale = min(prob_pura * peso, 1.0)

# Output dettagliato
print(f"Giocatore analizzato: {nome_giocatore.title()}")
print(f"Stato attuale (G{len(giornate)}): {stato_attuale} → {etichetta_stato[stato_attuale]}")
print(f"Probabilità pura (Markov): {prob_pura:.6f}")
print(f"Gol totali stagionali: {int(gol_totali)}")
print(f"Peso log(1+gol)/2.5: {peso:.6f}")
print(f"Probabilità finale (pura × peso): {prob_finale:.6f}")

# Matrici leggibili
df_trans = pd.DataFrame(trans.astype(int), columns=etichetta_stato, index=etichetta_stato)
df_prob = pd.DataFrame(np.round(matrice_prob, 3), columns=etichetta_stato, index=etichetta_stato)

from IPython.display import display, Markdown

display(Markdown(" **Matrice di transizione** (frequenze grezze con Laplace):"))
display(df_trans)

display(Markdown(" **Matrice di probabilità** (normalizzata):"))
display(df_prob)


Giocatore analizzato: Lautaro Martinez
Stato attuale (G38): 0 → 0 gol
Probabilità pura (Markov): 0.391304
Gol totali stagionali: 24
Peso log(1+gol)/2.5: 1.287550
Probabilità finale (pura × peso): 0.503824


 **Matrice di transizione** (frequenze grezze con Laplace):

,0 gol,1 gol,2+ gol
0 gol,14,7,2
1 gol,7,4,4
2+ gol,3,4,1


 **Matrice di probabilità** (normalizzata):

,0 gol,1 gol,2+ gol
0 gol,0.609,0.304,0.087
1 gol,0.467,0.267,0.267
2+ gol,0.375,0.500,0.125


Estrazione delle colonne '**Giocatore', 'Giornata', 'Prob_Gol**' e salvataggio in
**Probabilita_Storica.csv**

In [ ]:
from google.colab import files

# Carica il file completo
df = pd.read_csv("Probabilita_Storica_G4_G39.csv")

# Seleziona solo le prime 3 colonne
df_ridotto = df[['Giocatore', 'Giornata', 'Prob_Gol']]

# Salva il file ridotto
nome_file_ridotto = "Probabilita_Storica.csv"
df_ridotto.to_csv(nome_file_ridotto, index=False)

# Scarica il file
files.download(nome_file_ridotto)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 2. Calcolare la predizione goal (giornata per giornata)
### Obiettivo
Stimare il numero di gol attesi per ciascun giocatore dalla giornata 4 alla 39 di Serie A, utilizzando un modello di regressione basato su Decision Tree Regressor e sequenze storiche.

### Carica la matrice GolGiocatoriTemporale.csv

* Costruisce 3 modelli:
  * Modello G4 (finestra 3 + padding)
  * Modello G5 (finestra 4 + padding)
  * Modello globale G6+ (sliding window da 5 giornate)
* Predice i gol per ogni giornata da G4 a G39
* Combina la previsione con i gol stagionali normalizzati per ottenere la predizione pesata

### Output finale

File CSV: Predizione_Storica_G4_G39.csv
Contiene per ogni giocatore e giornata:

* Pred_Gol: previsione grezza
* Pred_Gol_Pesata: previsione pesata
* Gol_Stagionali e Gol_Normalizzati
* Analisi sui valori anomali (es. predizioni pari a 0.0)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Caricamento dati
df = pd.read_csv("GolGiocatoriTemporale.csv")
giornate = [col for col in df.columns if col.startswith("G") and col[1:].isdigit()]
dati = df[giornate].copy()
nomi = df['Giocatore'].values

# Parametri
window_size = 5
n_giornate = len(giornate)
giornata_max = n_giornate + 1  # fino a G39
tutte_le_righe = []

# Modello G4: training esteso con padding
X_g4, y_g4 = [], []
for _, row in dati.iterrows():
    for j in range(1, 4):  # G2, G3, G4
        finestra = row.iloc[max(0, j-3):j].values
        padding = np.zeros(3 - len(finestra))
        finestra = np.concatenate([padding, finestra])
        X_g4.append(finestra)
        y_g4.append(row.iloc[j])
model_g4 = DecisionTreeRegressor(random_state=42)
model_g4.fit(X_g4, y_g4)

# Modello G5: training esteso con padding
X_g5, y_g5 = [], []
for _, row in dati.iterrows():
    for j in range(1, 5):  # G2–G5
        finestra = row.iloc[max(0, j-4):j].values
        padding = np.zeros(4 - len(finestra))
        finestra = np.concatenate([padding, finestra])
        X_g5.append(finestra)
        y_g5.append(row.iloc[j])
model_g5 = DecisionTreeRegressor(random_state=42)
model_g5.fit(X_g5, y_g5)

# Modello G6+ (globale) con window da 5
X_train, y_train = [], []
for _, row in dati.iterrows():
    for i in range(window_size, n_giornate):
        X_train.append(row.iloc[i-window_size:i].values)
        y_train.append(row.iloc[i])
model_global = DecisionTreeRegressor(random_state=42)
model_global.fit(X_train, y_train)

# Predizioni da G4 a G39
for i in range(3, giornata_max):  # da G4 (index 3) a G39 (index 38)
    Gi = f"G{i+1}"
    inizio_finestra = max(0, i - window_size)
    fine_finestra = i
    print(f"Predizione per {Gi} usando G{inizio_finestra+1} → G{i}")

    # Estrai finestra
    X_test = dati.iloc[:, inizio_finestra:fine_finestra].values
    diff = window_size - X_test.shape[1]
    if diff > 0:
        padding = np.zeros((X_test.shape[0], diff))
        X_padded = np.hstack((padding, X_test))
    else:
        X_padded = X_test

    # Modello corretto
    if i == 3:
        y_pred = model_g4.predict(X_padded[:, -3:])
    elif i == 4:
        y_pred = model_g5.predict(X_padded[:, -4:])
    else:
        y_pred = model_global.predict(X_padded)

    # Gol cumulati fino a Gi−1
    giornate_fino = giornate[:i] if i <= n_giornate else giornate
    gol_stagionali = dati[giornate_fino].sum(axis=1)
    gol_norm = gol_stagionali / gol_stagionali.max()

    # Calcola predizione pesata finale
    pred_pesata = 0.7 * y_pred + 0.3 * gol_norm

    # Raccogli risultati
    for idx, nome in enumerate(nomi):
        tutte_le_righe.append({
            'Giocatore': nome,
            'Giornata': Gi,
            'Pred_Gol': round(y_pred[idx], 4),
            'Pred_Gol_Pesata': round(pred_pesata[idx], 4),
            'Gol_Stagionali': int(gol_stagionali.iloc[idx]),
            'Gol_Normalizzati': round(gol_norm.iloc[idx], 4)
        })

# Output finale
df_risultato = pd.DataFrame(tutte_le_righe)
df_risultato['Giornata_Num'] = df_risultato['Giornata'].str.extract(r'G(\d+)').astype(int)
df_risultato = df_risultato.sort_values(by=['Giornata_Num', 'Pred_Gol_Pesata'], ascending=[True, False]).reset_index(drop=True)
df_risultato.drop(columns=['Giornata_Num'], inplace=True)

# Salvataggio
ultima_giornata = df_risultato['Giornata'].iloc[-1]
nome_file = f"Predizione_Storica_G4_{ultima_giornata}.csv"
df_risultato.to_csv(nome_file, index=False)
print(f"\n File salvato come {nome_file}")

# Visualizza
from IPython.display import display
with pd.option_context('display.max_rows', None, 'display.precision', 6):
    for giornata in sorted(df_risultato['Giornata'].unique(), key=lambda g: int(g[1:])):
        print(f"\n Giornata: {giornata}")
        display(df_risultato[df_risultato['Giornata'] == giornata])

# Controlli su valori anomali nelle predizioni

print("\n ANALISI PRED_GOL == 0.0")

# Conteggio totale dei valori Pred_Gol_Pesata == 0.0
tot_zero = (df_risultato["Pred_Gol_Pesata"] == 0.0).sum()
print(f"\n Totale predizioni con valore 0.0: {tot_zero}")

# Conteggio per giornata
zero_pred_per_giornata = df_risultato[df_risultato["Pred_Gol_Pesata"] == 0.0]["Giornata"].value_counts()
zero_pred_per_giornata.index = zero_pred_per_giornata.index.str.extract(r'G(\d+)').astype(int)[0]
zero_pred_per_giornata = zero_pred_per_giornata.sort_index()

print("\n Pred_Gol_Pesata == 0.0 per giornata:")
for giornata, count in zero_pred_per_giornata.items():
    print(f"– G{giornata}: {count} casi")


Predizione per G4 usando G1 → G3
Predizione per G5 usando G1 → G4
Predizione per G6 usando G1 → G5
Predizione per G7 usando G2 → G6
Predizione per G8 usando G3 → G7
Predizione per G9 usando G4 → G8
Predizione per G10 usando G5 → G9
Predizione per G11 usando G6 → G10
Predizione per G12 usando G7 → G11
Predizione per G13 usando G8 → G12
Predizione per G14 usando G9 → G13
Predizione per G15 usando G10 → G14
Predizione per G16 usando G11 → G15
Predizione per G17 usando G12 → G16
Predizione per G18 usando G13 → G17
Predizione per G19 usando G14 → G18
Predizione per G20 usando G15 → G19
Predizione per G21 usando G16 → G20
Predizione per G22 usando G17 → G21
Predizione per G23 usando G18 → G22
Predizione per G24 usando G19 → G23
Predizione per G25 usando G20 → G24
Predizione per G26 usando G21 → G25
Predizione per G27 usando G22 → G26
Predizione per G28 usando G23 → G27
Predizione per G29 usando G24 → G28
Predizione per G30 usando G25 → G29
Predizione per G31 usando G26 → G30
Predizione per G

,Giocatore,Giornata,Pred_Gol,Pred_Gol_Pesata,Gol_Stagionali,Gol_Normalizzati
0,federico chiesa,G4,1.0000,0.8200,2,0.4
1,abdou harroui,G4,0.5000,0.4700,2,0.4
2,nicolas gonzalez,G4,0.5000,0.4700,2,0.4
3,christian pulisic,G4,0.5000,0.4700,2,0.4
4,dusan vlahovic,G4,0.5000,0.4700,2,0.4
5,andrea colpani,G4,0.3333,0.3533,2,0.4
6,domenico berardi,G4,0.2857,0.3200,2,0.4
7,gianluca scamacca,G4,0.2857,0.3200,2,0.4
8,lautaro martinez,G4,0.0000,0.3000,5,1.0
9,hakan calhanoglu,G4,0.3265,0.2886,1,0.2



 Giornata: G5


,Giocatore,Giornata,Pred_Gol,Pred_Gol_Pesata,Gol_Stagionali,Gol_Normalizzati
278,federico chiesa,G5,1.0000,0.8800,3,0.6
279,andrea pinamonti,G5,0.6667,0.6467,3,0.6
280,nemanja radonjicc,G5,0.6667,0.6467,3,0.6
281,giacomo bonaventura,G5,0.5000,0.4700,2,0.4
282,matteo politano,G5,0.5000,0.4700,2,0.4
283,mateo retegui,G5,0.5000,0.4700,2,0.4
284,domenico berardi,G5,0.4000,0.4000,2,0.4
285,cyril ngonge,G5,0.4000,0.4000,2,0.4
286,gianluca scamacca,G5,0.4000,0.4000,2,0.4
287,lautaro martinez,G5,0.0000,0.3000,5,1.0



 Giornata: G6


...


 Giornata: G38


,Giocatore,Giornata,Pred_Gol,Pred_Gol_Pesata,Gol_Stagionali,Gol_Normalizzati
9452,christian pulisic,G38,0.4000,0.4300,12,0.5000
9453,romelu lukaku,G38,0.3415,0.3890,12,0.5000
9454,gianluca scamacca,G38,0.3415,0.3765,11,0.4583
9455,lautaro martinez,G38,0.0897,0.3628,24,1.0000
9456,victor osimhen,G38,0.2000,0.3275,15,0.6250
9457,marcus thuram,G38,0.1875,0.2937,13,0.5417
9458,dusan vlahovic,G38,0.1142,0.2800,16,0.6667
9459,olivier giroud,G38,0.1111,0.2528,14,0.5833
9460,hakan calhanoglu,G38,0.1333,0.2433,12,0.5000
9461,milan djuric,G38,0.1818,0.2398,9,0.3750



 Giornata: G39


,Giocatore,Giornata,Pred_Gol,Pred_Gol_Pesata,Gol_Stagionali,Gol_Normalizzati
9730,nicolas gonzalez,G39,0.3750,0.4000,11,0.4583
9731,lautaro martinez,G39,0.1111,0.3778,24,1.0000
9732,olivier giroud,G39,0.2500,0.3750,16,0.6667
9733,rafael leao,G39,0.3415,0.3515,9,0.3750
9734,victor osimhen,G39,0.2182,0.3402,15,0.6250
9735,gianluca scamacca,G39,0.2500,0.3250,12,0.5000
9736,riccardo calafiori,G39,0.4000,0.3050,2,0.0833
9737,ademola lookman,G39,0.2000,0.2775,11,0.4583
9738,arthur,G39,0.3415,0.2640,2,0.0833
9739,romelu lukaku,G39,0.1471,0.2529,12,0.5000



 ANALISI PRED_GOL == 0.0

 Totale predizioni con valore 0.0: 0

 Pred_Gol_Pesata == 0.0 per giornata:


Estrazione delle colonne '**Giocatore', 'Giornata', 'Pred_Gol_Pesata**' e salvataggio in
**Predizione_Storica.csv.csv**

In [ ]:
# Estrazione delle colonne richieste
df_ridotto = df_risultato[['Giocatore', 'Giornata', 'Pred_Gol_Pesata']]

# Salvataggio file ridotto
nome_file_ridotto = f"Predizione_Storica.csv"
df_ridotto.to_csv(nome_file_ridotto, index=False)
print(f" File ridotto salvato come {nome_file_ridotto}")
from google.colab import files
files.download(nome_file_ridotto)


 File ridotto salvato come Predizione_Storica.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Andiamo ad unire Probabilità e Predizione Storiche in un unico file e lo salviamo

In [ ]:
# Carica i due file
df_prob = pd.read_csv("Probabilita_Storica.csv")
df_pred = pd.read_csv("Predizione_Storica.csv")

# Merge sul doppio campo 'Giocatore' + 'Giornata'
df_unito = pd.merge(df_prob, df_pred, on=["Giocatore", "Giornata"])

# Salva il file combinato
nome_file_unito = "Giocatori_Pred_Prob_Storiche.csv"
df_unito.to_csv(nome_file_unito, index=False)
print(f" File unito salvato come {nome_file_unito}")


 File unito salvato come Giocatori_Pred_Prob_Storiche.csv


In [ ]:

files.download(nome_file_unito)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>